# Part A: Extracting 150 frames

In [ ]:
import cv2
vidcap = cv2.VideoCapture('DragonBall.mkv')
success,image = vidcap.read()
count = 0
frame_count = 0
while success:
    if count%200 == 0:
        if frame_count <= 90:
            cv2.imwrite("frames/train/frame%d.jpg" % frame_count, image)     # save frame as JPEG file
        else:
            cv2.imwrite("frames/test/frame%d.jpg" % frame_count, image)     # save frame as JPEG file
        frame_count += 1
    success,image = vidcap.read()
    count += 1

    if frame_count >= 121:
        break

In [14]:
import numpy as np
import PIL
# Ground Truth
train = []
test = []

for i in range(0,90):
  I = np.asarray(PIL.Image.open('frames/train/frame'+str(i)+'.jpg'))
  train.append(I)
for i in range(91,121):
  I = np.asarray(PIL.Image.open('frames/test/frame'+str(i)+'.jpg'))
  test.append(I)

train = np.array(train)
test = np.array(test)

# Part B: Preprocessing


In [11]:
import cv2 as cv
from matplotlib import pyplot as plt

train_lineart = []
test_lineart = []

for i in range(0,90):
  I = cv.imread('frames/train/frame'+str(i)+'.jpg')
  edges = cv.Canny(I,100,200)
  train_lineart.append(edges)

for i in range(91,121):
  I = cv.imread('frames/train/frame'+str(i)+'.jpg')
  edges = cv.Canny(I,100,200)
  test_lineart.append(edges)

train_lineart = np.array(train_lineart)
test_lineart = np.array(test_lineart)

## Dataset Definition

In [ ]:
from os import listdir
from os.path import join, exists
from PIL import Image
import numpy as np
import torch
import torch.utils.data as data
import torchvision.transforms as transforms

from skimage import io, feature, color, img_as_uint, util
from skimage.transform import resize

class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir):
        super(DatasetFromFolder, self).__init__()
        self.photo_path = image_dir
        self.image_filenames = [x for x in listdir(self.photo_path) if is_image_file(x)]
        transform_list = [transforms.ToTensor()]
        self.transform = transforms.Compose(transform_list)

    def __getitem__(self, index):
        # Load Image
        try:
            target_path = join(self.photo_path, self.image_filenames[index])
            frame_num = target_path.split("e")[-1]
            frame_num = int(frame_num.split(".")[0]) - 1
            #will be either black or colored
            frame_prev = self.get_prev(frame_num) 
            target = load_img(target_path)
            input = color.rgb2gray(target)
            #Lineart
            input = feature.canny(input,sigma = 1)
            input = util.invert(input)
            input = Image.fromarray(np.uint8(input)*255)
            
            input = Image.fromarray(input)
            frame_prev = self.transform(frame_prev)
            target = self.transform(target)
            input = self.transform(input)

            return input, target, frame_prev
        except:
            print("Something went wrong frame:" + str(frame_num))
            return self[0]

    def __len__(self):
        return len(self.image_filenames)

    def get_prev(self, num):
        if not exists(join(self.photo_path,"frame"+str(num)+".jpg")):
            initial_prev_frame = Image.new("RGB",[256,256])
            return initial_prev_frame
        else:
            #define rnd num generator and if statement <0.5 take black or color
            rnd = np.random.uniform(0,1)
            if rnd <= 0.5:
                prev = load_img(join(self.photo_path,"frame"+str(num)+".jpg"))
            else:
                prev = Image.new("RGB",[256,256])

            return prev
            

In [16]:
from os.path import join
from torch.utils.data import DataLoader

def get_training_set(root_dir):
    train_dir = join(root_dir, "Train")

    return DatasetFromFolder(train_dir)


def get_test_set(root_dir):
    test_dir = join(root_dir, "Test")

    return DatasetFromFolder(test_dir)

def get_val_set(root_dir):
    val_dir = join(root_dir, "Val")

    return DatasetFromFolder(val_dir)

def create_iterator(sample_size, sample_dataset):
    while True:
        sample_loader = DataLoader(
            dataset= sample_dataset,
            batch_size=sample_size,
            drop_last=True
        )

        for item in sample_loader:
            yield item


## Defining Loss Functions

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class AdversarialLoss(nn.Module):

    def __init__(self, type='nsgan', target_real_label=1.0, target_fake_label=0.0):
        super(AdversarialLoss, self).__init__()

        self.type = type
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))

        if type == 'nsgan':
            self.criterion = nn.BCELoss() 
    

        elif type == 'lsgan':
            self.criterion = nn.MSELoss()

        elif type == 'hinge':
            self.criterion = nn.ReLU()

    def __call__(self, outputs, is_real, is_disc=None):
        if self.type == 'hinge':
            if is_disc:
                if is_real:
                    outputs = -outputs
                return self.criterion(1 + outputs).mean()
            else:
                return (-outputs).mean()

        else:
            labels = (self.real_label if is_real else self.fake_label).expand_as(outputs)
            loss = self.criterion(outputs, labels)
            return loss


class StyleLoss(nn.Module):
    def __init__(self):
        super(StyleLoss, self).__init__()
        self.add_module('vgg', VGG19())
        self.criterion = torch.nn.L1Loss()

    def compute_gram(self, x):
        b, ch, h, w = x.size()
        f = x.view(b, ch, w * h)
        f_T = f.transpose(1, 2)
        G = f.bmm(f_T) / (h * w * ch)

        return G

    def __call__(self, x, y):
        # Compute features
        x_vgg, y_vgg = self.vgg(x), self.vgg(y)

        # Compute loss
        style_loss = 0.0
        style_loss += self.criterion(self.compute_gram(x_vgg['relu2_2']), self.compute_gram(y_vgg['relu2_2']))
        style_loss += self.criterion(self.compute_gram(x_vgg['relu3_4']), self.compute_gram(y_vgg['relu3_4']))
        style_loss += self.criterion(self.compute_gram(x_vgg['relu4_4']), self.compute_gram(y_vgg['relu4_4']))
        style_loss += self.criterion(self.compute_gram(x_vgg['relu5_2']), self.compute_gram(y_vgg['relu5_2']))

        return style_loss



class PerceptualLoss(nn.Module):

    def __init__(self, weights=[1.0, 1.0, 1.0, 1.0, 1.0]):
        super(PerceptualLoss, self).__init__()
        self.add_module('vgg', VGG19())
        self.criterion = torch.nn.L1Loss()
        self.weights = weights

    def __call__(self, x, y):
        x_vgg, y_vgg = self.vgg(x), self.vgg(y)

        content_loss = 0.0
        content_loss += self.weights[0] * self.criterion(x_vgg['relu1_1'], y_vgg['relu1_1'])
        content_loss += self.weights[1] * self.criterion(x_vgg['relu2_1'], y_vgg['relu2_1'])
        content_loss += self.weights[2] * self.criterion(x_vgg['relu3_1'], y_vgg['relu3_1'])
        content_loss += self.weights[3] * self.criterion(x_vgg['relu4_1'], y_vgg['relu4_1'])
        content_loss += self.weights[4] * self.criterion(x_vgg['relu5_1'], y_vgg['relu5_1'])


        return content_loss

## Importing Helper Functions

In [ ]:
import helper_function

In [5]:
def sample(iteration):
    with torch.no_grad():

        input,target,prev_frame = next(sample_iterator)
        
        if opt.cuda:
            input = input.cuda()
            target = target.cuda()
            prev_frame = prev_frame.cuda()

        pred_input = torch.cat((input,prev_frame),1)
        prediction = netG(pred_input)
        prediction = postprocess(prediction)
        input = postprocess(input)
        target = postprocess(target)

    img = stitch_images(input, target, prediction)
    samples_dir = root_path + "/samples"

    if not os.path.exists(samples_dir):
        os.makedirs(samples_dir)

    sample = opt.dataset + "_" + str(epoch) + "_" + str(iteration).zfill(5) + ".png"
    print('\nsaving sample ' + sample + ' - learning rate: ' + str(opt.lr))
    img.save(os.path.join(samples_dir, sample))

def log_train_data(loginfo):
    log_dir = root_path + "/logs"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    log_file = log_dir + "/" + opt.logfile
    with open(log_file, 'a') as f:
        f.write('%s\n' % ' '.join([str(item[1]) for item in loginfo]))


def checkpoint(epoch):
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")
    if not os.path.exists(os.path.join("checkpoint", opt.dataset)):
        os.mkdir(os.path.join("checkpoint", opt.dataset))
    net_g_model_out_path = "checkpoint/{}/netG_weights_epoch_{}.pth".format(opt.dataset, epoch)
    net_d_model_out_path = "checkpoint/{}/netD_weights_epoch_{}.pth".format(opt.dataset, epoch)
    
    torch.save({'generator': netG.state_dict()}, net_g_model_out_path)
    torch.save({'discriminator': netD.state_dict()}, net_d_model_out_path)
    
    print("Checkpoint saved to {}".format("checkpoint" + opt.dataset))

## Training Parameters

In [2]:
opt = {
    'root': './',
    'dataset': 'frames/',
    'logfile': True,
    'checkpoint_path_G':True,
    'checkpoint_path_D':True,
    'batchSize':16,
    'testBatchSize':1,
    'nEpochs':100,
    'input_nc':1,
    'output_nc':3,
    'lr':0.0001,
    'beta1':0,
    'cuda':'store_true',
    'threads':0,
    'seed':123,
    'L1lamb':10,
    'Stylelamb':1000,
    'Contentlamb':1,
    'Adversariallamb':0.1
}

In [ ]:
from __future__ import print_function
import argparse
import os
import numpy as np
from math import log10
from os.path import join
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms


os.environ['CUDA_VISIBLE_DEVICES'] = '0'




if opt.cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

cudnn.benchmark = True

torch.manual_seed(opt.seed)
if opt.cuda:
    torch.cuda.manual_seed(opt.seed)

print('===> Loading datasets')
root_path = opt.root
train_set = get_training_set(join(root_path , opt.dataset))
test_set = get_test_set(join(root_path , opt.dataset))

training_data_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)
testing_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.testBatchSize, shuffle=False)

sample_iterator = create_iterator(6, test_set)

print('===> Building model')
netG = define_G(opt.input_nc, opt.output_nc, opt.ngf, False, [0])
netD = define_D(opt.input_nc + opt.output_nc, opt.ndf, False, [0])

if opt.checkpoint_path_G and opt.checkpoint_path_D:
    load(opt.checkpoint_path_G, opt.checkpoint_path_D, netG, netD)

criterionGAN = AdversarialLoss()
criterionSTYLE = StyleLoss()
criterionCONTENT = PerceptualLoss()
criterionL1 = nn.L1Loss()
criterionMSE = nn.MSELoss()

# setup optimizer
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr * 0.1, betas=(opt.beta1, 0.999))

print('---------- Networks initialized -------------')
print_network(netG)
print_network(netD)
print('-----------------------------------------------')

real_a = torch.FloatTensor(opt.batchSize, opt.input_nc, 256, 256)
real_b = torch.FloatTensor(opt.batchSize, opt.output_nc, 256, 256)
prev_b = torch.FloatTensor(opt.batchSize, opt.output_nc, 256, 256)

if opt.cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    criterionGAN = criterionGAN.cuda()
    criterionL1 = criterionL1.cuda()
    critertionSTYLE = criterionSTYLE.cuda()
    criterionCONTENT = criterionCONTENT.cuda()
    criterionMSE = criterionMSE.cuda()
    real_a = real_a.cuda()
    real_b = real_b.cuda()
    prev_b = prev_b.cuda()

real_a = Variable(real_a)
real_b = Variable(real_b)
prev_b = Variable(prev_b)

def train(epoch):

    for iteration, batch in enumerate(training_data_loader, 1):
        # forward
        real_a_cpu, real_b_cpu, prev_b_cpu = batch[0], batch[1], batch[2]
        real_a.data.resize_(real_a_cpu.size()).copy_(real_a_cpu)
        real_b.data.resize_(real_b_cpu.size()).copy_(real_b_cpu)
        prev_b.data.resize_(prev_b_cpu.size()).copy_(prev_b_cpu)

        input_joined = torch.cat((real_a,prev_b),1)

        fake_b = netG(input_joined)

        ############################
        # (1) Update D network: maximize log(D(x,y)) + log(1 - D(x,G(x)))
        ###########################

        optimizerD.zero_grad()

        # train with fake
        fake_ab = torch.cat((real_a, prev_b, fake_b), 1)
        pred_fake = netD.forward(fake_ab.detach())
        loss_d_fake = criterionGAN(pred_fake,False,True)

        # train with real
        real_ab = torch.cat((real_a, prev_b, real_b), 1)
        pred_real = netD.forward(real_ab)
        loss_d_real = criterionGAN(pred_real, True, True) 


        # Combined loss
        loss_d = (loss_d_fake + loss_d_real) * 0.5

        loss_d.backward()

        #Discriminator parameters update every 12 iterations 
        if (iteration == 1 or iteration % 12 == 0):
            optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(x,G(x))) + L1(y,G(x))
        ##########################
        optimizerG.zero_grad()

        # First, G(A) should fake the discriminator
        fake_ab = torch.cat((real_a, prev_b,fake_b), 1)
        pred_fake = netD.forward(fake_ab)
        loss_g_gan = criterionGAN(pred_fake, True, False)

        # Second, G(A) = B
        loss_g_l1 = criterionL1(fake_b, real_b) * opt.L1lamb
        loss_g = loss_g_gan + loss_g_l1

        loss_g_style = criterionSTYLE(fake_b,real_b) * opt.Stylelamb
        loss_g = loss_g + loss_g_style

        loss_g_content = criterionCONTENT(fake_b,real_b) * opt.Contentlamb
        loss_g = loss_g + loss_g_content

        loss_g.backward()

        optimizerG.step()

        if (iteration % 25 == 0):
            logs = [("epoc", epoch),("iter", iteration),("Loss_G", loss_g.item()),("Loss_D", loss_d.item()), ("Loss_G_adv",loss_g_gan.item()),("Loss_G_L1",loss_g_l1.item()),("Loss_G_style",loss_g_style.item()),("Loss_G_content",loss_g_content.item()),("Loss_D_Real",loss_d_real.item()),("Loss_D_Fake",loss_d_fake.item())]
            log_train_data(logs)

        if(iteration % 250 == 0):
            sample(iteration)


        print("===> Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f} LossD_Fake: {:.4f} LossD_Real: {:.4f}  LossG_Adv: {:.4f} LossG_L1: {:.4f} LossG_Style {:.4f} LossG_Content {:.4f}".format(
           epoch, iteration, len(training_data_loader), loss_d, loss_g, loss_d_fake, loss_d_real, loss_g_gan, loss_g_l1, loss_g_style, loss_g_content))

def sample(iteration):
    with torch.no_grad():

        input,target,prev_frame = next(sample_iterator)
        
        if opt.cuda:
            input = input.cuda()
            target = target.cuda()
            prev_frame = prev_frame.cuda()

        pred_input = torch.cat((input,prev_frame),1)
        prediction = netG(pred_input)
        prediction = postprocess(prediction)
        input = postprocess(input)
        target = postprocess(target)

    img = stitch_images(input, target, prediction)
    samples_dir = root_path + "/samples"

    if not os.path.exists(samples_dir):
        os.makedirs(samples_dir)

    sample = opt.dataset + "_" + str(epoch) + "_" + str(iteration).zfill(5) + ".png"
    print('\nsaving sample ' + sample + ' - learning rate: ' + str(opt.lr))
    img.save(os.path.join(samples_dir, sample))

def log_train_data(loginfo):
    log_dir = root_path + "/logs"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    log_file = log_dir + "/" + opt.logfile
    with open(log_file, 'a') as f:
        f.write('%s\n' % ' '.join([str(item[1]) for item in loginfo]))


def checkpoint(epoch):
    if not os.path.exists("checkpoint"):
        os.mkdir("checkpoint")
    if not os.path.exists(os.path.join("checkpoint", opt.dataset)):
        os.mkdir(os.path.join("checkpoint", opt.dataset))
    net_g_model_out_path = "checkpoint/{}/netG_weights_epoch_{}.pth".format(opt.dataset, epoch)
    net_d_model_out_path = "checkpoint/{}/netD_weights_epoch_{}.pth".format(opt.dataset, epoch)
    
    torch.save({'generator': netG.state_dict()}, net_g_model_out_path)
    torch.save({'discriminator': netD.state_dict()}, net_d_model_out_path)
    
    print("Checkpoint saved to {}".format("checkpoint" + opt.dataset))

for epoch in range(1, opt.nEpochs + 1):
    train(epoch)
    checkpoint(epoch)


def run():
    torch.multiprocessing.freeze_support()
    print('loop')

if __name__ == '__main__':
    run()

## Defining Loss And Optimizer

In [ ]:
if opt.cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")

cudnn.benchmark = True

torch.manual_seed(opt.seed)
if opt.cuda:
    torch.cuda.manual_seed(opt.seed)

print('===> Loading datasets')
root_path = opt.root
train_set = get_training_set(join(root_path , opt.dataset))
test_set = get_test_set(join(root_path , opt.dataset))

training_data_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)
testing_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.testBatchSize, shuffle=False)

sample_iterator = create_iterator(6, test_set)

netG = define_G(opt.input_nc, opt.output_nc, opt.ngf, False, [0])
netD = define_D(opt.input_nc + opt.output_nc, opt.ndf, False, [0])

if opt.checkpoint_path_G and opt.checkpoint_path_D:
    load(opt.checkpoint_path_G, opt.checkpoint_path_D, netG, netD)

criterionGAN = AdversarialLoss()
criterionSTYLE = StyleLoss()
criterionCONTENT = PerceptualLoss()
criterionL1 = nn.L1Loss()
criterionMSE = nn.MSELoss()

# setup optimizer
optimizerG = optim.Adam(netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=opt.lr * 0.1, betas=(opt.beta1, 0.999))

In [ ]:
real_a = torch.FloatTensor(opt.batchSize, opt.input_nc, 256, 256)
real_b = torch.FloatTensor(opt.batchSize, opt.output_nc, 256, 256)
prev_b = torch.FloatTensor(opt.batchSize, opt.output_nc, 256, 256)

if opt.cuda:
    netD = netD.cuda()
    netG = netG.cuda()
    criterionGAN = criterionGAN.cuda()
    criterionL1 = criterionL1.cuda()
    critertionSTYLE = criterionSTYLE.cuda()
    criterionCONTENT = criterionCONTENT.cuda()
    criterionMSE = criterionMSE.cuda()
    real_a = real_a.cuda()
    real_b = real_b.cuda()
    prev_b = prev_b.cuda()

real_a = Variable(real_a)
real_b = Variable(real_b)
prev_b = Variable(prev_b)

## Training

In [4]:
def train(epoch):

    for iteration, batch in enumerate(training_data_loader, 1):
        # forward
        real_a_cpu, real_b_cpu, prev_b_cpu = batch[0], batch[1], batch[2]
        real_a.data.resize_(real_a_cpu.size()).copy_(real_a_cpu)
        real_b.data.resize_(real_b_cpu.size()).copy_(real_b_cpu)
        prev_b.data.resize_(prev_b_cpu.size()).copy_(prev_b_cpu)

        input_joined = torch.cat((real_a,prev_b),1)

        fake_b = netG(input_joined)

        ############################
        # (1) Update D network: maximize log(D(x,y)) + log(1 - D(x,G(x)))
        ###########################

        optimizerD.zero_grad()

        # train with fake
        fake_ab = torch.cat((real_a, prev_b, fake_b), 1)
        pred_fake = netD.forward(fake_ab.detach())
        loss_d_fake = criterionGAN(pred_fake,False,True)

        # train with real
        real_ab = torch.cat((real_a, prev_b, real_b), 1)
        pred_real = netD.forward(real_ab)
        loss_d_real = criterionGAN(pred_real, True, True) 


        # Combined loss
        loss_d = (loss_d_fake + loss_d_real) * 0.5

        loss_d.backward()

        #Discriminator parameters update every 12 iterations 
        if (iteration == 1 or iteration % 12 == 0):
            optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(x,G(x))) + L1(y,G(x))
        ##########################
        optimizerG.zero_grad()

        # First, G(A) should fake the discriminator
        fake_ab = torch.cat((real_a, prev_b,fake_b), 1)
        pred_fake = netD.forward(fake_ab)
        loss_g_gan = criterionGAN(pred_fake, True, False)

        # Second, G(A) = B
        loss_g_l1 = criterionL1(fake_b, real_b) * opt.L1lamb
        loss_g = loss_g_gan + loss_g_l1

        loss_g_style = criterionSTYLE(fake_b,real_b) * opt.Stylelamb
        loss_g = loss_g + loss_g_style

        loss_g_content = criterionCONTENT(fake_b,real_b) * opt.Contentlamb
        loss_g = loss_g + loss_g_content

        loss_g.backward()

        optimizerG.step()

        if (iteration % 25 == 0):
            logs = [("epoc", epoch),("iter", iteration),("Loss_G", loss_g.item()),("Loss_D", loss_d.item()), ("Loss_G_adv",loss_g_gan.item()),("Loss_G_L1",loss_g_l1.item()),("Loss_G_style",loss_g_style.item()),("Loss_G_content",loss_g_content.item()),("Loss_D_Real",loss_d_real.item()),("Loss_D_Fake",loss_d_fake.item())]
            log_train_data(logs)

        if(iteration % 250 == 0):
            sample(iteration)


        print("===> Epoch[{}]({}/{}): Loss_D: {:.4f} Loss_G: {:.4f} LossD_Fake: {:.4f} LossD_Real: {:.4f}  LossG_Adv: {:.4f} LossG_L1: {:.4f} LossG_Style {:.4f} LossG_Content {:.4f}".format(
           epoch, iteration, len(training_data_loader), loss_d, loss_g, loss_d_fake, loss_d_real, loss_g_gan, loss_g_l1, loss_g_style, loss_g_content))


In [ ]:
for epoch in range(1, opt.nEpochs + 1):
    train(epoch)
    checkpoint(epoch)

## Testing the model

In [ ]:
# Testing settings
opt = {
    'root': './',
    'dataset': 'frames/',
    'model': 'checkpoint/facades/netG_model_epoch_200.pth',
    'cuda':'store_true'
}

root_path = opt.root
val_set = get_val_set(os.path.join(root_path , opt.dataset))

seq_sampler = SequentialSampler(val_set)

val_data_loader = DataLoader(dataset=val_set, num_workers=0, batch_size=1, shuffle=False,sampler = seq_sampler)

checkpoint = torch.load(opt.model)
netG = InpaintGenerator()
netG.load_state_dict(checkpoint['generator'])
netG.cuda()

transform_list = [transforms.ToTensor(),
                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]

transform = transforms.Compose(transform_list)

counter = 0

with torch.no_grad():
    for batch in val_data_loader:
        input, target, prev_frame = Variable(batch[0], volatile=True), Variable(batch[1], volatile=True), Variable(batch[2], volatile=True)
        if opt.cuda:
            input = input.cuda()
            target = target.cuda()
            prev_frame = prev_frame.cuda()
        if counter != 0:
            prev_frame = tmp
            print("success")
        pred_input = torch.cat((input,prev_frame),1)
        out = netG(pred_input)
        tmp = out
        
        if not os.path.exists(os.path.join("results", opt.dataset)):
            os.makedirs(os.path.join("results", opt.dataset))
       
        image_name = opt.dataset + "_" + str(counter).zfill(5) + ".jpg"
        save_image(out,"results/{}/{}".format(opt.dataset, image_name))
        print("saving:"+image_name)
        
        counter += 1